<a href="https://colab.research.google.com/github/DrEduardo1337/Data_Mine_degree/blob/main/Dados_Alemanha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import pandas as pd
from google.colab import drive

In [23]:
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/dados/sagah.csv')

print(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   Biomass  Fossil Brown coal/Lignite  Fossil Coal-derived gas  Fossil Gas  \
0   4719.0                    13270.0                    534.0      1165.0   
1   4710.0                    13344.0                    526.0      1163.0   
2   4706.0                    13285.0                    525.0      1154.0   
3   4703.0                    13293.0                    517.0      1148.0   
4   4701.0                    13694.0                    518.0      1159.0   

   Fossil Hard coal  Fossil Oil  Fossil Oil shale  Fossil Peat  Geothermal  \
0            4439.0         0.0               0.0          0.0         6.0   
1            4460.0         0.0               0.0          0.0         6.0   
2            4594.0         0.0               0.0          0.0         6.0   
3            4648.0         0.0               0.0          0.0         6.0   
4           

In [24]:
# Analisar Dados Ausentes
# Verificar se há algum valor nulo
missing_values_count = df.isnull().sum()

print("Contagem de valores faltantes por coluna:")
print(missing_values_count[missing_values_count > 0])

# Para verificar quais dias têm dados faltantes, precisamos primeiro garantir que a coluna 'Date' seja do tipo datetime.
# Converter a coluna 'Date' para datetime, se ainda não for
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Identificar as linhas com dados faltantes
rows_with_missing_data = df[df.isnull().any(axis=1)]

# Obter os dias únicos com dados faltantes
days_with_missing_data = rows_with_missing_data['Date'].dt.date.unique()

print("\nDias com dados faltantes:")
if len(days_with_missing_data) > 0:
    for day in days_with_missing_data:
        print(day)
else:
    print("Não há dias com dados faltantes.")

# Contar o número total de dias únicos no dataset
total_unique_days = df['Date'].dt.date.nunique()

# Contar o número de dias com dados completos
# Assumindo que um dia completo tem 24 horas * (60/15 minutos por registro) = 96 registros
# Primeiro, vamos agrupar por data e contar o número de registros por dia
records_per_day = df.groupby(df['Date'].dt.date).size()

# Dias com dados completos são aqueles que têm 96 registros
days_with_complete_data_count = (records_per_day == 96).sum()

print(f"\nNúmero total de dias únicos no dataset: {total_unique_days}")
print(f"Número de dias com dados completos (assumindo 96 registros por dia): {days_with_complete_data_count}")
print(f"Número de dias com dados faltantes: {len(days_with_missing_data)}")

Contagem de valores faltantes por coluna:
Biomass                              1
Fossil Gas                           2
Fossil Hard coal                     2
Fossil Oil                          96
Fossil Oil shale                   181
Fossil Peat                        181
Hydro Pumped Storage                 2
Hydro Run-of-river and poundage      2
Marine                             181
Nuclear                              2
Other renewable                      2
Solar                                1
Emission                            99
Consumption                          9
Generation                         280
dtype: int64

Dias com dados faltantes:
2017-03-12
2017-03-26
2017-05-16
2017-06-24
2017-06-29
2017-06-30
2018-01-07
2018-01-08
2018-01-19
2018-02-04
2018-02-13
2018-02-17
2018-04-26
2018-07-30
2018-08-01

Número total de dias únicos no dataset: 730
Número de dias com dados completos (assumindo 96 registros por dia): 726
Número de dias com dados faltantes: 15


In [26]:
# Fontes de Energia por Geração Média
# Identificar as colunas que representam fontes de energia.
# Assumindo que todas as colunas, exceto 'Date', 'Hour', 'Emission', 'Consumption', e 'Generation' são fontes de energia.
energy_columns = df.columns.difference(['Date', 'Hour', 'Emission', 'Consumption', 'Generation'])

# Calcula média de geração para cada fonte de energia
average_generation_per_source = df[energy_columns].mean()

# Encontrar a fonte de energia com a maior média de geração
most_generated_source = average_generation_per_source.idxmax()
max_average_generation = average_generation_per_source.max()

print("\nMédia de Geração por Fonte de Energia:")
print(average_generation_per_source.sort_values(ascending=False))

print(f"\nFonte de energia com a maior média de geração: {most_generated_source}")
print(f"Média de geração: {max_average_generation:.2f}")


Média de Geração por Fonte de Energia:
Fossil Brown coal/Lignite          14724.595049
Wind Onshore                        9973.386615
Nuclear                             8221.482662
Fossil Hard coal                    7869.442407
Biomass                             4588.779049
Solar                               4401.507342
Fossil Gas                          2898.159793
Wind Offshore                       2082.995491
Hydro Run-of-river and poundage     1674.416436
Hydro Pumped Storage                1101.917891
Waste                                624.678624
Fossil Coal-derived gas              428.670519
Other renewable                      168.714390
Fossil Oil                           158.146905
Hydro Water Reservoir                 86.158390
Geothermal                             3.771062
Fossil Oil shale                       0.000000
Fossil Peat                            0.000000
Marine                                 0.000000
dtype: float64

Fonte de energia com a maior méd

In [27]:
# Analisar Fontes de Energia em Relação ao Consumo Médio
# Calcular o consumo médio
average_consumption = df['Consumption'].mean()

# Calcular 5% do consumo médio
threshold = 0.05 * average_consumption

print(f"\nConsumo médio: {average_consumption:.2f}")
print(f"5% do consumo médio (limite): {threshold:.2f}")

# Identificar as fontes de energia cuja média de geração é maior que o limite
sources_above_threshold = average_generation_per_source[average_generation_per_source > threshold]

print("\nFontes de energia com média de geração maior que 5% do consumo médio:")
print(sources_above_threshold.sort_values(ascending=False))


Consumo médio: 57185.17
5% do consumo médio (limite): 2859.26

Fontes de energia com média de geração maior que 5% do consumo médio:
Fossil Brown coal/Lignite    14724.595049
Wind Onshore                  9973.386615
Nuclear                       8221.482662
Fossil Hard coal              7869.442407
Biomass                       4588.779049
Solar                         4401.507342
Fossil Gas                    2898.159793
dtype: float64


In [28]:
# Analisar Fontes de Energia Renovável (Wind Offshore, Wind Onshore, Solar)
# Calcular a média de geração para as fontes de energia renovável especificadas
renewable_sources = ['Wind Offshore', 'Wind Onshore', 'Solar']
average_generation_renewable = df[renewable_sources].mean()

print("\nMédia de Geração para Fontes de Energia Renovável (Wind Offshore, Wind Onshore, Solar):")
print(average_generation_renewable.sort_values(ascending=False))

# Identificar a fonte de energia renovável com a maior média de geração entre as selecionadas
most_generated_renewable = average_generation_renewable.idxmax()
max_average_generation_renewable = average_generation_renewable.max()

print(f"\nFonte de energia renovável com a maior média de geração (entre Wind Offshore, Wind Onshore, Solar): {most_generated_renewable}")
print(f"Média de geração: {max_average_generation_renewable:.2f}")


Média de Geração para Fontes de Energia Renovável (Wind Offshore, Wind Onshore, Solar):
Wind Onshore     9973.386615
Solar            4401.507342
Wind Offshore    2082.995491
dtype: float64

Fonte de energia renovável com a maior média de geração (entre Wind Offshore, Wind Onshore, Solar): Wind Onshore
Média de geração: 9973.39
